# Stochastic Gradient Descent
You should build an end-to-end machine learning pipeline using a stochastic gradient descent model. In particular, you should do the following:
- Load the `mnist` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the datasets folder.
- Split the dataset into training and test sets using [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
- Build an end-to-end machine learning pipeline, including a [stochastic gradient descent](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [5]:
df=pd.read_csv('/mnist.csv')
df.head()

,id,class,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,31953,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34452,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60897,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,36953,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1981,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.isnull().sum()

,0
id,0
class,0
pixel1,0
pixel2,0
pixel3,0
...,...
pixel780,0
pixel781,0
pixel782,0
pixel783,0


In [7]:
X = df.drop(['id'], axis=1)
y = df['id']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

In [9]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline

In [11]:
base_clf = make_pipeline(
    StandardScaler(),
    SGDClassifier(max_iter=1000, tol=1e-3, random_state=42))

In [19]:
y_train = y_train.ravel()
y_test  = y_test.ravel()

<ipython-input-19-14d183f85772>:1: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
<ipython-input-19-14d183f85772>:2: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test  = y_test.ravel()


In [12]:
base_clf.fit(X_train, y_train)
print("Train score:", base_clf.score(X_train, y_train))
print("Test score: ", base_clf.score(X_test, y_test))

Train score: 0.8959375
Test score:  0.0


In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {
  'standardscaler': [StandardScaler()],      # could also try MinMaxScaler()
  'sgdclassifier__loss': ['hinge', 'log', 'modified_huber'],
  'sgdclassifier__penalty': ['l2', 'l1', 'elasticnet'], # avoid overfitting
  'sgdclassifier__alpha': [1e-4, 1e-3, 1e-2, 1e-1], #coefficient for penaöty
  'sgdclassifier__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
  'sgdclassifier__eta0': [1e-3, 1e-2, 1e-1]
}
#maxiter increase the number of steps to reach the point of the minimum

In [15]:
pipe = make_pipeline(StandardScaler(), SGDClassifier(random_state=42))

In [16]:
grid = GridSearchCV(
  pipe,
  param_grid=param_grid,
  cv=5,
  scoring='accuracy',    # or 'f1_macro', 'roc_auc_ovr', …
  n_jobs=-1,
  verbose=1
)

In [17]:
grid.fit(X_train, y_train)
print("Best CV score:", grid.best_score_)
print("Best params:  ", grid.best_params_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


ValueError: n_splits=5 cannot be greater than the number of members in each class.

In [ ]:
best_clf = grid.best_estimator_
test_score = best_clf.score(X_test, y_test)
print("Test accuracy of best model:", test_score)